In [ ]:
#Projet Mogpl M1 Informatique
#Dice Battle

#Yuhan WANG & Yufei LIU

In [126]:
import numpy as np
import matplotlib.pyplot as plt
import math

#variable global N est le point cible et D est le nombre de dés qu'on peut lancer maximument chaque fois
#LP = ligne perdu signifie que si joueur1 a moins de cette ligne et joueur 2 a N-1 point alors joueur1 a perdu
N = 100
D = 10
LP = N - 6*D

#Q1
def Q(d,k):
    if k<=0 | ((k>1) & (k<2*d)) | k>6*d :
        return 0
    elif d==1:
        if k>1 & k<7:
            return 1/5
        else:
            return 0
    else:
        res = 0
        for j in range(2,7):
            res += Q(d-1,k-j)
        return res/5


def calcul_tableau_P(d,k):
    values_possibles = [ i for i in range(1,k+1) ]
    res = np.zeros((d , k))
    #d2 = 2*d
    for i in range(d):
        for j in range(k):
            ii = i+1
            jj = j+1
            if jj == 1:
                res[i][j] = 1 - (5/6)**ii
            elif jj < 2*ii:
                res[i][j] = 0
            else:
                res[i][j] = (5/6)**ii * Q(ii,jj)
    return res
    
print(calcul_tableau_P(5,20))


[[1.66666667e-01 1.66666667e-01 1.66666667e-01 1.66666667e-01
  1.66666667e-01 1.66666667e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.05555556e-01 0.00000000e+00 0.00000000e+00 2.77777778e-02
  5.55555556e-02 8.33333333e-02 1.11111111e-01 1.38888889e-01
  1.11111111e-01 8.33333333e-02 5.55555556e-02 2.77777778e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.21296296e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.62962963e-03 1.38888889e-02 2.77777778e-02
  4.62962963e-02 6.94444444e-02 8.33333333e-02 8.79629630e-02
  8.33333333e-02 6.94444444e-02 4.62962963e-02 2.77777778e-02
  1.38888889e-02 4.62962963e-03 0.00000000e+00 0.00000000e+00]
 [5.17746914e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.0

In [142]:
tab_P = calcul_tableau_P(10,100)

0.16666666666666663


In [147]:
print(tab_P[9][0])

0.8384944171101543


In [29]:
#Q2

def calcul_EP(d):
    return 4*d*(5/6)**d + 1 - (5/6)**d

print(calcul_EP(5))
print(calcul_EP(6))
#On doit choisir d*=6 car EP(6) est supérieur de EP(5)




8.63567386831276
8.702653463648836


In [164]:
#Q4
np.set_printoptions(threshold=np.inf)

def EG(i,j):
    lim = N-1+6*D
    if (i>=N) | ((i==N-1) & (j<N)):
        return 1
    elif (i<LP & j==N-1) | j>=N:
        return -1
    else:
        tab = np.zeros(((lim-i+1),(lim-j+1)))
                
        for a in range(lim-i+1):
            for b in range(lim-j+1):
                if(a<6*D):
                    tab[a][b]=1
                elif(a==6*D):
                    if(b>=6*D):
                        tab[a][b]=1
                    else:
                        tab[a][b]=-1
                elif((6*D<a)&(b<6*D)):
                    tab[a][b]=-1
        #print("heelo",tab)
        for c in range(6*D+1,lim-i+1):
            for d in range(6*D,lim-j+1):
                if tab[c][d]==0:
                    res = (1/6)**2 * tab[c-1][d-1]
                    for e in range(2*D,6*D+1):
                        res += tab[c-e][d-1] / (4*D+1) * (1/6)*(5/6)
                        res += tab[c-1][d-e] / (4*D+1) * (1/6)*(5/6)
                        for f in range(2*D,6*D+1):
                            res += tab[c-e][d-f] / (4*D+1)**2 * (5/6)**2
                    tab[c][d]=res
        #print("bye",tab)
        return tab[lim-i][lim-j]
        
              
            

    


In [215]:
#Q4
np.set_printoptions(threshold=np.inf)

def EG(i,j):
    lim = N-1+6*D
    if (i>=N) | ((i==N-1) & (j<N)):
        return 1
    elif (i<LP & j==N-1) | j>=N:
        return -1
    else:
        tab = np.zeros(((lim-i+1),(lim-j+1)))
                
        for a in range(lim-i+1):
            for b in range(lim-j+1):
                if(a<6*D):
                    tab[a][b]=1
                elif(a==6*D):
                    if(b>=6*D):
                        tab[a][b]=1
                    else:
                        tab[a][b]=-1
                elif((6*D<a)&(b<6*D)):
                    tab[a][b]=-1
        #print("heelo",tab)
        for c in range(6*D+1,lim-i+1):
            for d in range(6*D,lim-j+1):
                if tab[c][d]==0:
                    cas_j1 = []
                    for d1 in range(1,D+1):
                        res1 = 0
                        for e1 in range(1,6*d1+1):
                            cas_j2 = []
                            for d2 in range(1,D+1):
                                res2 = 0
                                for e2 in range(1,6*d2+1):
                                    res2 += tab[c-e1][d-d2] * tab_P[d2-1][e2-1]
                                cas_j2.append(res2)
                            opt_j2 = np.argmin(np.array(cas_j2))
                            res1 += cas_j2[opt_j2] * tab_P[d1-1][e1-1]
                        cas_j1.append(res1)
                tab[c][d]=max(cas_j1)  
        #print("bye",tab.shape,tab)
        return tab[lim-i][lim-j]
        
              
            

    



In [165]:
print(EG(99,1))
print(EG(98,1))
print(EG(97,1))
print(EG(96,1))
print(EG(95,1))
print(EG(50,50))
print(EG(20,20))
print(EG(0,0))
print(EG(5,40))

1
0.9999999999999786
0.9941934548071523
0.9901037548277974
0.9896624713762225
0.36717526293443087
0.31685084226803284
0.3029416684874429
-0.31470565855743793


In [225]:
[EG(i,99) for i in range(1,100)]








[-0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999999999999,
 -0.9999999669236564,
 -0.9999996361602227,
 -0.9999978169613369,
 -0.9999905401657944,
 -0.9999668905802807,
 -0.9999010025042759,
 -0.999738

In [ ]:
#Q7

def strategie_aveugle():
    return 6


#def strategie_optimale(i,j):
    